Import libraries

In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import time

Check TensorFlow Version

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.2.1


Check for a GPU

In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /gpu:0


Define load_vgg()

In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    # Load the saved model
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    
    # Get the tensor layers by name
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

Run test

In [5]:
tests.test_load_vgg(load_vgg, tf)

Tests Passed


Define layers()

In [6]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # Here we will use FCN-8 architecture developed at Berkeley. (https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf)
    # Here is the encoder architecture
    # conv7 = Do convolution on layer 7
    # upsampled_conv7 = Upsample conv7
    # conv4 = Do convolution on layer 4
    # skip4 = Connect upsampled_conv7 to conv4
    # upsampled_skip4 = Upsample skip4
    # conv3 = Do convolution on layer 3
    # skip3 = Connect upsampled_skip4 to conv3
    # upsampled_skip3 = Upsample skip3
    # output = upsampled_skip3

    # Set standard deviation of weights
    weights_stddev = 0.01
    
    # Set L2 regularizer of weights
    weights_l2_regularizer = 1e-3
    
    # Do 1x1 convolution on vgg16 layer 7
    conv7 = tf.layers.conv2d(vgg_layer7_out, filters = num_classes, kernel_size = 1, strides = (1,1), padding = 'same',
                             kernel_initializer = tf.random_normal_initializer(stddev = weights_stddev),
                             kernel_regularizer = tf.contrib.layers.l2_regularizer(weights_l2_regularizer)
                            )
    
    # Do unsample on vgg16 layer 7
    upsampled_conv7 = tf.layers.conv2d_transpose(conv7, filters = num_classes, kernel_size = 4, strides = (2, 2), padding = 'same',
                                                 kernel_initializer = tf.random_normal_initializer(stddev = weights_stddev),
                                                 kernel_regularizer = tf.contrib.layers.l2_regularizer(weights_l2_regularizer)
                                                )
    
    # Do 1x1 convolution on vgg16 layer 4
    conv4 = tf.layers.conv2d(vgg_layer4_out, filters = num_classes, kernel_size = 1, strides = (1,1), padding = 'same',
                             kernel_initializer = tf.random_normal_initializer(stddev = weights_stddev),
                             kernel_regularizer = tf.contrib.layers.l2_regularizer(weights_l2_regularizer)
                            )
    
    # Do skip connection between unsampled_cov7 and conv4
    skip4 = tf.add(upsampled_conv7, conv4)

    # Do unsample on skip4
    upsampled_skip4 = tf.layers.conv2d_transpose(skip4, filters = num_classes, kernel_size = 4, strides = (2, 2), padding = 'same',
                                                 kernel_initializer = tf.random_normal_initializer(stddev = weights_stddev),
                                                 kernel_regularizer = tf.contrib.layers.l2_regularizer(weights_l2_regularizer)
                                                )
    
    # Do 1x1 convolution on vgg16 layer 3
    conv3 = tf.layers.conv2d(vgg_layer3_out, filters = num_classes, kernel_size = 1, strides = (1,1), padding = 'same',
                             kernel_initializer = tf.random_normal_initializer(stddev = weights_stddev),
                             kernel_regularizer = tf.contrib.layers.l2_regularizer(weights_l2_regularizer)
                            )
    
    # Do skip connection between unsampled_skip4 and conv3
    skip3 = tf.add(upsampled_skip4, conv3)

    # Do unsample on skip3
    upsampled_skip3 = tf.layers.conv2d_transpose(skip3, filters = num_classes, kernel_size = 16, strides = (8, 8), padding = 'same',
                                                 kernel_initializer = tf.random_normal_initializer(stddev = weights_stddev),
                                                 kernel_regularizer = tf.contrib.layers.l2_regularizer(weights_l2_regularizer)
                                                )
    
    # Output is the unsampled_skip3
    output = upsampled_skip3
    
    return output

Run test

In [7]:
tests.test_layers(layers)

Tests Passed


Define optimize()

In [8]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    # Remember the output tensor is 4D so we have to reshape it to 2D
    # logits is now a 2D tensor where each row represents a pixel and each column a class.
    logits = tf.reshape(nn_last_layer, (-1, num_classes)) ## Remove this line???
    
    # Reshape correct_label tensor to 2D
    labels = tf.reshape(correct_label, (-1, num_classes))
    
    # We can just use standard cross entropy loss function
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = labels, logits = logits))
    
    # Use Adam optimizer for training
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss

Run test

In [9]:
tests.test_optimize(optimize)

Tests Passed


Define train_nn()

In [10]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    
    # Run global variables initializer
    sess.run(tf.global_variables_initializer())
    
    # Start training
    print("Training...")
    print()
    
    for epoch in range(epochs):
        # Print result for record
        print("EPOCH {} ...".format(epoch+1))
        start_time = time.time()
        
        for image, label in get_batches_fn(batch_size):
            # Training
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict = {input_image: image, correct_label: label,
                                            keep_prob: 0.5, learning_rate: 0.00001
                                           }
                              )
            print("Loss = {:.3f}".format(loss))
        elapsed_time = time.time() - start_time
        print("Elapsed time = {:.3f}".format(elapsed_time))
        print()
    
    # Finish training
    print("Training finished.")

Run test

In [11]:
tests.test_train_nn(train_nn)

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


Define run()

In [12]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        
        # Create placeholders
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name = 'correct_label')
        learning_rate = tf.placeholder(tf.float32, name = 'learning_rate')
        
        # Load the layers from the VGG16
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        
        # Construct new layers
        output_layer = layers(layer3_out, layer4_out, layer7_out, num_classes)

        # TODO: Train NN using the train_nn function
        # Define optimizer
        logits, train_op, cross_entropy_loss = optimize(output_layer, correct_label, learning_rate, num_classes)
        
        # Define training epochs and batch size
        epochs = 40
        batch_size = 5
        
        # print('Before training')
        
        # Start training
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image, correct_label, keep_prob, learning_rate)
        
        # print('After training')

        print('Before saving inference data')
        
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        print('After saving inference data')
        
        # OPTIONAL: Apply the trained model to a video


Run training

In [13]:
if __name__ == '__main__':
    run()